In [1]:
# Get Quantization levels
# 8 bit int can represent for 256 levels
import tensorflow as tf
import time
import os
import numpy as np

import labelreg.helpers as helper
import labelreg.networks as network
import labelreg.utils as util
import labelreg.losses as loss
NUM_OF_LEVEL = 256

def get_quantization_levels(model_meta, model_dir):
    tf.reset_default_graph()

    with tf.Session() as sess:
        # load the checkpoint
        sess.run(tf.global_variables_initializer())
        saver = tf.train.import_meta_graph(model_meta)
        saver.restore(sess, model_dir)
        graph = tf.get_default_graph()
        all_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        quant_levels = {}
        for v in all_vars:
            # quantize only network variables
            if ('fcn' in v.name) and ('Adam' not in v.name):
                level_name = v.name
                v_numpy = sess.run(v)
                v_numpy_shape = v_numpy.shape
                v_numpy_flatten = v_numpy.flatten()
                min_np = v_numpy.min()
                max_np = v_numpy.max()
                gap = (max_np - min_np) / (NUM_OF_LEVEL - 1)
                quant_levels[level_name] = np.asarray([(min_np + x * gap) for x in range(NUM_OF_LEVEL)])
    return quant_levels
    
# one method to do quantization
def strategy_1(v_numpy):
    v_numpy_shape = v_numpy.shape
    v_numpy_flatten = v_numpy.flatten()
    min_np = v_numpy.min()
    max_np = v_numpy.max()
    gap = (max_np - min_np) / (NUM_OF_LEVEL - 1)
    levels = np.asarray([(min_np + x * gap) for x in range(NUM_OF_LEVEL)])
    for i in range(len(v_numpy_flatten)):
        v_numpy_flatten[i] = min(levels, key=lambda x : abs(x-v_numpy_flatten[i]))
    v_numpy = v_numpy_flatten.reshape(v_numpy_shape)
    return v_numpy


def quantize_one_variable(sess, v,levels):
    v_numpy = sess.run(v)
    v_numpy_shape = v_numpy.shape
    v_numpy_flatten = v_numpy.flatten()
    for i in range(len(v_numpy_flatten)):
        v_numpy_flatten[i] = min(levels, key=lambda x : abs(x-v_numpy_flatten[i]))
    v_numpy = v_numpy_flatten.reshape(v_numpy_shape)
    new_v = v_numpy
    assign_op = v.assign(new_v)
    return assign_op

def quantize_model(model_meta,model_dir,target_dir,quant_levels):
    tf.reset_default_graph()

    with tf.Session() as sess:
        # load the checkpoint
        sess.run(tf.global_variables_initializer())
        saver = tf.train.import_meta_graph(model_meta)
        saver.restore(sess, model_dir)
        graph = tf.get_default_graph()
        all_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        update_operation = []
        for v in all_vars:
            # quantize only network variables
            if ('fcn' in v.name) and ('Adam' not in v.name):
                level_name = v.name
                levels = quant_levels[level_name]
                update_operation.append(quantize_one_variable(sess, v,levels))
        _ = sess.run(update_operation)
        # save the qunatized checkpoint
        saver.save(sess, target_dir)

In [2]:
#Quantize Model
model_dir = r"C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\pruned_model_retrain2\model.ckpt"
model_meta = r"C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\pruned_model_retrain2\model.ckpt.meta"
target_dir = r"C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\quantized_model2\model.ckpt"
QUANT_LEVELS = get_quantization_levels(model_meta,model_dir)
quantize_model(model_meta,model_dir,target_dir,QUANT_LEVELS)

INFO:tensorflow:Restoring parameters from C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\pruned_model_retrain2\model.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\pruned_model_retrain2\model.ckpt


In [3]:
# inference quantized model
import tensorflow as tf
import sys

import labelreg.helpers as helper
import labelreg.networks as network
import labelreg.apps as app


tf.reset_default_graph()
# 0 - get configs
config = helper.ConfigParser(sys.argv, 'inference')

# 1 - images to register
reader_moving_image, reader_fixed_image, _, _ = helper.get_data_readers(config['Inference']['dir_moving_image'],
                                                                        config['Inference']['dir_fixed_image'])

# 2 - graph
# network for predicting ddf only
ph_moving_image = tf.placeholder(tf.float32, [reader_moving_image.num_data]+reader_moving_image.data_shape+[1])
ph_fixed_image = tf.placeholder(tf.float32, [reader_fixed_image.num_data]+reader_fixed_image.data_shape+[1])

reg_net = network.build_network(network_type=config['Network']['network_type'],
                                minibatch_size=reader_moving_image.num_data,
                                image_moving=ph_moving_image,
                                image_fixed=ph_fixed_image)

# restore the trained weights
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, config['Inference']['file_model_saved'])


# 3 - compute ddf
testFeed = {ph_moving_image: reader_moving_image.get_data(),
            ph_fixed_image: reader_fixed_image.get_data()}
ddf = sess.run(reg_net.ddf, feed_dict=testFeed)
helper.write_images(ddf, config['Inference']['dir_save'], 'ddf')

# warp the test images
warped_images = app.warp_volumes_by_ddf(reader_moving_image.get_data(), ddf)
helper.write_images(warped_images, config['Inference']['dir_save'], 'warped_image')

# warp test labels of gland segmentation, i.e. label_indices=0
if config['Inference']['dir_moving_label']:
    data_moving_label = helper.DataReader(config['Inference']['dir_moving_label'],labelr=True).get_data(label_indices=[1])
    warped_labels = app.warp_volumes_by_ddf(data_moving_label, ddf)
    helper.write_images(warped_labels, config['Inference']['dir_save'], 'warped_label')


Reading default config file in: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\config_demo.ini.

[Network]: network_type: local
[Inference]: file_model_saved: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\quantized_model2\model.ckpt
[Inference]: dir_moving_image: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\moving_images
[Inference]: dir_fixed_image: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\fixed_images
[Inference]: dir_save: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\inference_results\Quantized2_results
[Inference]: dir_moving_label: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\moving_labels
[Inference]: dir_fixed_label: C: